In [ ]:
from datetime import datetime, timedelta
from openbb_terminal import api
import os

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
from IPython.display import display

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
api.theme.applyMPLstyle()

In [ ]:
interval_opts = [
    "1m",
    "2m",
    "5m",
    "15m",
    "30m",
    "60m",
    "90m",
    "1h",
    "1d",
    "5d",
    "1wk",
    "1mo",
    "3mo",
]
def clean_str(string):
    new_str = ""
    for letter in string:
        if letter.isupper():
            new_str += " "
        new_str += letter
    return new_str.title()


def format_plotly(fig, data, start, end, chart, calc=None):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    start_t = start.strftime("%Y/%m/%d")
    end_t = end.strftime("%Y/%m/%d")
    if calc:
        if len(calc) == 1:
            fig_title = f"{calc[0]} of {data} from {start_t} to {end_t}"
        else:
            fig_title = f"{', '.join(calc)} of {data} from {start_t} to {end_t}"
    else:
        fig_title = "Volume"
    height = 500 if chart == "main" else 300
    fig.update_layout(
        margin=dict(l=0, r=10, t=10, b=10),
        autosize=False,
        width=900,
        height=height,
        legend=dict(orientation="h"),
        title={
            "text": fig_title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )


def format_df(df: pd.DataFrame):
    df.reset_index(inplace=True)
    df.columns = [x.lower() for x in df.columns]

In [ ]:
class Chart:
    def __init__(self):
        self.last_tickers = ""
        self.last_interval = "1d"
        self.df = pd.DataFrame()
        self.infos = {}

    def create_stock(
        self, calculation, start, end, interval, tickers, target_column
    ):
        if tickers and tickers[-1] == ",":
            if tickers != self.last_tickers or interval != self.last_interval:
                if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                    self.df = yf.download(
                        tickers, period="max", interval=interval, progress=False
                    )
                else:
                    end_date = end + timedelta(days=1)
                    self.df = yf.download(
                        tickers,
                        start=start,
                        end=end_date,
                        interval=interval,
                        progress=False,
                    )
                format_df(self.df)
                self.last_tickers = tickers
                self.last_interval = interval

            start_n = datetime(start.year, start.month, start.day)
            end_n = datetime(end.year, end.month, end.day)
            fig = go.Figure()
            calcs = self.df
            if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                result = calcs.loc[
                    (calcs["date"] >= start_n) & (calcs["date"] <= end_n)
                ]
            else:
                result = calcs
            if not target_column:
                target_column = self.df.columns[0]
            api.forecast.expo(result, target_column="close", n_predict=5)

In [ ]:
w_auto = widgets.Layout(width="auto")
calc_widget = widgets.SelectMultiple(
    options=[1,2,3,4], value=[1], layout=w_auto
)

base_date = (datetime.today() - timedelta(days=365)).date()
start_widget = widgets.DatePicker(value=base_date, layout=w_auto, description="Start")
end_widget = widgets.DatePicker(
    value=datetime.today().date(), layout=w_auto, description="End"
)

target_widget = widgets.Dropdown(
    options=[], layout=w_auto, description="Target"
)

interval_widget = widgets.Dropdown(
    options=interval_opts, value="1d", layout=w_auto, description="Interval"
)
tickers_widget = widgets.Textarea(
    value="TSLA,", layout=widgets.Layout(width="auto", height="100%")
)
selection_box = widgets.VBox([tickers_widget, target_widget])
date_box = widgets.VBox([start_widget, end_widget, interval_widget])
controls = widgets.HBox(
    [selection_box, calc_widget, date_box],
    layout=widgets.Layout(width="90%"),
)
chart = Chart()
stocks_view = widgets.interactive_output(
    chart.create_stock,
    {
        "calculation": calc_widget,
        "start": start_widget,
        "end": end_widget,
        "interval": interval_widget,
        "tickers": tickers_widget,
        "target_column": target_widget,
    },
)

title_html = "<h1>Timeseries Forecasting Dashboard</h1>"

app_contents = [widgets.HTML(title_html), controls, stocks_view]
app = widgets.VBox(app_contents)
display(app)